In [170]:
import pandas as pd
import numpy as np

In [171]:
dataset = pd.read_csv('AmesHousing.csv')

In [172]:
for col in dataset.columns:
  count = dataset[col].isnull().sum()
  print(col, count)

Order 0
PID 0
MS SubClass 0
MS Zoning 0
Lot Frontage 490
Lot Area 0
Street 0
Alley 2732
Lot Shape 0
Land Contour 0
Utilities 0
Lot Config 0
Land Slope 0
Neighborhood 0
Condition 1 0
Condition 2 0
Bldg Type 0
House Style 0
Overall Qual 0
Overall Cond 0
Year Built 0
Year Remod/Add 0
Roof Style 0
Roof Matl 0
Exterior 1st 0
Exterior 2nd 0
Mas Vnr Type 1775
Mas Vnr Area 23
Exter Qual 0
Exter Cond 0
Foundation 0
Bsmt Qual 80
Bsmt Cond 80
Bsmt Exposure 83
BsmtFin Type 1 80
BsmtFin SF 1 1
BsmtFin Type 2 81
BsmtFin SF 2 1
Bsmt Unf SF 1
Total Bsmt SF 1
Heating 0
Heating QC 0
Central Air 0
Electrical 1
1st Flr SF 0
2nd Flr SF 0
Low Qual Fin SF 0
Gr Liv Area 0
Bsmt Full Bath 2
Bsmt Half Bath 2
Full Bath 0
Half Bath 0
Bedroom AbvGr 0
Kitchen AbvGr 0
Kitchen Qual 0
TotRms AbvGrd 0
Functional 0
Fireplaces 0
Fireplace Qu 1422
Garage Type 157
Garage Yr Blt 159
Garage Finish 159
Garage Cars 1
Garage Area 1
Garage Qual 159
Garage Cond 159
Paved Drive 0
Wood Deck SF 0
Open Porch SF 0
Enclosed Porch 0
3Ssn

In [173]:


dataset.drop(columns=['Roof Matl','Heating','Condition 2','Utilities','Order','PID','Alley', 'Misc Feature', 'Fence', 'Pool QC', 'Mas Vnr Type', 'Fireplace Qu'], inplace=True)

In [174]:
X = dataset.drop('SalePrice', axis=1)
y = dataset['SalePrice'].values

In [175]:
numerical_cols = X.select_dtypes(include=['number']).columns
categorical_cols = X.select_dtypes(include=['object']).columns

In [176]:
# Imputer for numerical: Mean
from sklearn.impute import SimpleImputer
num_imputer = SimpleImputer(strategy='mean')
X[numerical_cols] = num_imputer.fit_transform(X[numerical_cols])

# Imputer for categorical: Most Frequent (mode)
cat_imputer = SimpleImputer(strategy='most_frequent')
X[categorical_cols] = cat_imputer.fit_transform(X[categorical_cols])


In [177]:
categorical_cols = X.select_dtypes(include=['object']).columns
binary_cols = [col for col in categorical_cols if X[col].nunique(dropna=False) == 2]
multi_valued_cols = [col for col in categorical_cols if X[col].nunique(dropna=False) > 2]

print('Binary Columns:')
print(binary_cols)

print('multi_valued_cols:')
print(multi_valued_cols)





Binary Columns:
['Street', 'Central Air']
multi_valued_cols:
['MS Zoning', 'Lot Shape', 'Land Contour', 'Lot Config', 'Land Slope', 'Neighborhood', 'Condition 1', 'Bldg Type', 'House Style', 'Roof Style', 'Exterior 1st', 'Exterior 2nd', 'Exter Qual', 'Exter Cond', 'Foundation', 'Bsmt Qual', 'Bsmt Cond', 'Bsmt Exposure', 'BsmtFin Type 1', 'BsmtFin Type 2', 'Heating QC', 'Electrical', 'Kitchen Qual', 'Functional', 'Garage Type', 'Garage Finish', 'Garage Qual', 'Garage Cond', 'Paved Drive', 'Sale Type', 'Sale Condition']


In [178]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
for col in binary_cols:
    X[col] = le.fit_transform(X[col])

In [179]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

ct = ColumnTransformer([('encode', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), multi_valued_cols)], remainder='passthrough')



In [180]:
X = ct.fit_transform(X)
print(X)

print(y)


[[0.000e+00 0.000e+00 0.000e+00 ... 0.000e+00 5.000e+00 2.010e+03]
 [0.000e+00 0.000e+00 0.000e+00 ... 0.000e+00 6.000e+00 2.010e+03]
 [0.000e+00 0.000e+00 0.000e+00 ... 1.250e+04 6.000e+00 2.010e+03]
 ...
 [0.000e+00 0.000e+00 0.000e+00 ... 7.000e+02 7.000e+00 2.006e+03]
 [0.000e+00 0.000e+00 0.000e+00 ... 0.000e+00 4.000e+00 2.006e+03]
 [0.000e+00 0.000e+00 0.000e+00 ... 0.000e+00 1.100e+01 2.006e+03]]
[215000 105000 172000 ... 132000 170000 188000]


In [181]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)